# TMCS Chemical Informatics Workshop 1

## Data Cleaning

This notebook will detail the process of the data cleaning undertaken on the provided solubility data.

In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('../data/raw/solubility_data_raw.csv')
raw_data.head()

,CAS #,Formula,Mol Weight,Chemical Name,Solubility S/ (mg/L),LogP,Melting Point / C,SMILES
0,000055-91-4,C6H14FO3P,184.15,ISOFLUROPHATE,15400.0,1.13,-82,O=P(F)(OC(C)C)OC(C)C
1,000056-23-5,CCl4,153.82,CARBON TETRACHLORIDE,793.0,2.83,250,C(Cl)(Cl)(Cl)Cl
2,000052-24-4,C6H12N3PS,189.22,THIOTEPA,190000.0,0.53,51.5,S=P(N1CC1)(N2CC2)N3CC3
3,000052-68-6,C4H8Cl3O4P,257.44,TRICHLORFON,120000.0,0.51,77,COP(=O)(OC)C(O)C(Cl)(Cl)Cl
4,000053-19-0,C14H10Cl4,320.05,"O,P'-DDD",0.1,5.87,77,ClC(Cl)C(c1ccc(Cl)cc1)c2ccccc2Cl


Let's look at the data types as suggested.

In [3]:
for colname in raw_data.columns.values:
    print(colname, raw_data[colname].dtype)

CAS # object
Formula object
Mol Weight float64
Chemical Name object
Solubility S/ (mg/L) float64
LogP float64
Melting Point / C object
SMILES object


The melting point is weird - we would expect numerical values? What's going on?

In [4]:
raw_data['Melting Point / C']

0            -82
1            250
2           51.5
3             77
4             77
5          129.1
6          134.5
7          146.5
8          151.5
9            157
10           158
11         170.5
12         171.4
13           172
14           173
15           174
16           180
17           222
18         260.2
19           263
20         269.5
21       138-139
22    140-141   
Name: Melting Point / C, dtype: object

The last two variables have been given as ranges - we can't do modelling with this! Let's change it so we're using the mean value.

In [5]:
import copy
import numpy as np

processed_data = copy.deepcopy(raw_data) # It is best to keep the raw data totally unchanged

processed_data.loc[21, 'Melting Point / C'] = 138.5
processed_data.loc[22, 'Melting Point / C'] = 140.5

# Now lets change the data type to a float. This will also see if our process has worked!
processed_data['Melting Point / C'] = processed_data['Melting Point / C'].astype(np.float64)

print(processed_data['Melting Point / C'])

0     -82.0
1     250.0
2      51.5
3      77.0
4      77.0
5     129.1
6     134.5
7     146.5
8     151.5
9     157.0
10    158.0
11    170.5
12    171.4
13    172.0
14    173.0
15    174.0
16    180.0
17    222.0
18    260.2
19    263.0
20    269.5
21    138.5
22    140.5
Name: Melting Point / C, dtype: float64


Now we can save this data. I will save it as `imputed_data.csv` as we have filled in some weird values.

In [6]:
processed_data.to_csv('../data/processed/imputed_data.csv', index=False) # So we dont get a phantom column

Next, I can see that I need to use the $Log(S)$ descriptor, so I will calculate it here. I will do the same to convert from Celsius to Kelvin for the melting point

In [7]:
# Creation of logS
processed_data['LogS'] = np.log(processed_data['Solubility S/ (mg/L)'])

# Conversion to Kelvin
processed_data['Melting Point / K'] = processed_data['Melting Point / C'] + 273

I'll then save this file too.

In [8]:
processed_data.to_csv('../data/processed/transformed_data.csv', index=False) # So we dont get a phantom column

Lets delete any rows with missing values lastly.

In [9]:
processed_data.dropna(axis=0, inplace=True)

In [10]:
processed_data.to_csv('../data/processed/processed_data.csv')